# Build End-to-End LLM project for a retail domain (t-shirts selling store).

Tech stack for this project
* UI: Streamlit
* LLM: Google Gemini LLM model
* Embeddings: Hugging Face
* Framework: Langchain

# Importing required modules

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain import hub
from dotenv import load_dotenv
import os
import getpass

# Insert API key

In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Initializing Gemini AI model

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Establishing database connection

In [4]:
db_user = "root"
db_password = "Kitsune#sql98"
db_host = "localhost"
db_name = "atliq_tshirts"

In [5]:
connection_Uri = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"
db = SQLDatabase.from_uri(connection_Uri)

In [6]:
print(db.table_info)
print("--------")
print("--------")
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM discounts LIMIT 5;")}')


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

# Core functionality

## SQL Query Generation

In [7]:
"""
1. Pulls a specialized SQL generation prompt from LangChain Hub
2. Formats the prompt with database schema information
3. Sends the prompt to Gemini AI for query generation
4. Uses a second prompt to extract just the SQL from the response
"""

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

def write_query(question: str):
    """Generate SQL query from the user's question."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": question,
        }
    )
    response = llm.invoke(prompt.to_string())
    extraction_prompt = """
    Please extract the SQL query from the following text and return only the SQL query without any additional characters or formatting:

    {response}

    SQL Query:
    """
    # Format the prompt with the actual response
    prompt = extraction_prompt.format(response=response)
    # Invoke the language model with the prompt
    parsed_query = llm.invoke(prompt)
    return parsed_query.content

c:\Users\danyc\Desktop\llm_sqldb\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Query Execution

In [8]:
"""
This function creates a query execution tool and runs the generated SQL against your database, returning the raw results.
"""

def execute_query(query: str):
    """Execute the SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return execute_query_tool.invoke(query)

## Natural Language Answer Generation

In [9]:
"""
This function takes the original question, generated SQL, and query results, then asks Gemini to formulate a human-friendly answer.
"""

def generate_answer(question: str, query: str, result: str):
    """Generate an answer using the query results."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {question}\n'
        f'SQL Query: {query}\n'
        f'SQL Result: {result}'
    )
    response = llm.invoke(prompt)
    return response.content

# Putting It All Together

In [35]:
def model_output(input):
    query = write_query(input)
    result = execute_query(query)
    answer = generate_answer(input, query, result)

    print(f"Query: \n\n{query}")
    print("\n")
    print(answer)

In [36]:
# question = "How many t-shirts do we have left for nike in extra small size and white color?"
# question = "How much is the price of the inventory for all small size t-shirts?"
# question = "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
question = "If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"

model_output(question)

Query: 

SELECT
  SUM(
    (
      t.price * (
        1 - (
          d.pct_discount / 100
        )
      )
    ) * t.stock_quantity
  ) AS revenue
FROM t_shirts AS t
JOIN discounts AS d
  ON t.t_shirt_id = d.t_shirt_id
WHERE
  t.brand = 'Van Huesen';


Answer: The store will generate $9634.25 in revenue if all Van Heusen T-shirts are sold today with the discounts applied.
